<a href="https://colab.research.google.com/github/abeSanchez/Machine-Learning-UCF/blob/master/homework1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning Homework 1

In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.utils import to_categorical
from keras import models
from keras import layers

(x_train, y_train), (x_test, y_test) = mnist.load_data()

Using TensorFlow backend.


In [0]:
def sigmoid(z):
  return 1.0 / (1.0 + np.exp(-z))

def sigmoid_prime(z):
  sz = sigmoid(z)
  return sz * (1 - sz)

def se_loss(a, y):
  return 0.5 * (a - y) ** 2

def se_gradient(a, x, y):
  return (a - y) * sigmoid_prime(a) * x

def bce_loss(a, y):
  return -y * np.log10(a + 1e-9) - (1 - y) * np.log10(1 - a + 1e-9)

def bce_gradient(a, x, y):
  return (a - y) * x

def cce_loss(a, y):
  return -np.sum(y * np.log10(a + 1e-9))

def cce_gradient(a, x, y):
  return (a - y) * x

def softmax(z):
  exps = np.exp(z - np.max(z))
  return exps / np.sum(exps)

In [0]:
# Flatten images and apply feature scaling
flat_x_train = x_train.reshape(60000, 28 * 28) / 255
flat_x_test = x_test.reshape(10000, 28 * 28) / 255

In [0]:
# Create new array for labels for each digit
y_train_single_digit = {}
y_test_single_digit = {}

for i in range(10):
  y_train_single_digit[i] = y_train == i
  y_train_single_digit[i] = y_train_single_digit[i].astype(int)
  y_test_single_digit[i] = y_test == i
  y_test_single_digit[i] = y_test_single_digit[i].astype(int)

In [0]:
# One hot encoding for labels
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [0]:
# Create dictionaries for storing the weights
w_mse = {}
w_bce = {}

for i in range(10):
  w_mse[i] = (np.random.rand(28 * 28, 1).reshape((784,)) - 0.5, np.random.rand(1))
  w_bce[i] = (np.random.rand(28 * 28, 1).reshape((784,)) - 0.5, np.random.rand(1))
  
w_cce = np.random.rand(28 * 28 * 10, 1).reshape((784,10)) - 0.5 
w_cce_b = np.random.rand(10, 1)

# Problem 1
## Using logistic regression with mean squared error loss

### Training the classifiers

In [0]:
epochs = 20
lr = 0.05
batch_size = 32

for digit in range(10):
  for epoch in range(epochs):
    shuffled_indices = np.random.permutation(len(flat_x_train))
    flat_x_train_shuffled = flat_x_train[shuffled_indices]
    y_train_single_digit_shuffled = y_train_single_digit[digit][shuffled_indices]
    
    for i in range(0, len(flat_x_train), batch_size):
      xi = flat_x_train_shuffled[i:i+batch_size]
      yi = y_train_single_digit_shuffled[i:i+batch_size].reshape(32,1)
      z = xi.dot(w_mse[digit][0]) + w_mse[digit][1]
      a = sigmoid(z).reshape(32,1)
      gradient = np.sum(se_gradient(a, xi, yi), axis=0) / batch_size
      gradient_b = np.sum(se_gradient(a, 1, yi), axis=0) / batch_size
      w_mse[digit] = (w_mse[digit][0] - lr * gradient, w_mse[digit][1] - lr * gradient_b)

### Testing the classifiers

In [8]:
for digit in range(10):
  loss = 0
  correct = 0

  for example in range(len(flat_x_test)):
      z = flat_x_test[example].dot(w_mse[digit][0]) + w_mse[digit][1]
      a = sigmoid(z)
      prediction = int(round(a[0]))
      loss += se_loss(prediction, y_test_single_digit[digit][example])
      correct += 1 if y_test_single_digit[digit][example] == prediction else 0
        
  loss /= len(flat_x_test)
  accuracy = correct / len(flat_x_test)
  print("Classifier " + str(digit) + " | loss: " + str(loss) + " - accuracy: " + str(accuracy))

Classifier 0 | loss: 0.0041 - accuracy: 0.9918
Classifier 1 | loss: 0.00375 - accuracy: 0.9925
Classifier 2 | loss: 0.0106 - accuracy: 0.9788
Classifier 3 | loss: 0.0117 - accuracy: 0.9766
Classifier 4 | loss: 0.0103 - accuracy: 0.9794
Classifier 5 | loss: 0.01405 - accuracy: 0.9719
Classifier 6 | loss: 0.00765 - accuracy: 0.9847
Classifier 7 | loss: 0.0079 - accuracy: 0.9842
Classifier 8 | loss: 0.0208 - accuracy: 0.9584
Classifier 9 | loss: 0.0184 - accuracy: 0.9632


### Using all classifiers to perform digit classification on test data

In [9]:
correct = 0

for example in range(len(flat_x_test)):
  y_predict = np.array([])  
  
  for classifier in range(10):
    z = flat_x_test[example].dot(w_mse[classifier][0]) + w_mse[classifier][1]
    a = sigmoid(z)
    y_predict = np.append(y_predict, a)
    
  if np.argmax(y_predict) == y_test[example]:
    correct += 1
    
print("Accuracy on test data : " + str(correct / len(flat_x_test)))

Accuracy on test data : 0.9109


# Problem 2
## Using logistic regression with binary cross entropy loss

### Training the classifiers

In [0]:
epochs = 20
lr = 0.05
batch_size = 32

for digit in range(10):
  for epoch in range(epochs):
    shuffled_indices = np.random.permutation(len(flat_x_train))
    flat_x_train_shuffled = flat_x_train[shuffled_indices]
    y_train_single_digit_shuffled = y_train_single_digit[digit][shuffled_indices]
    
    for i in range(0, len(flat_x_train), batch_size):
      xi = flat_x_train_shuffled[i:i+batch_size]
      yi = y_train_single_digit_shuffled[i:i+batch_size].reshape(32,1)
      z = xi.dot(w_bce[digit][0]) + w_bce[digit][1]
      a = sigmoid(z).reshape(32,1)
      gradient = np.sum(bce_gradient(a, xi, yi), axis=0) / batch_size
      gradient_b = np.sum(bce_gradient(a, 1, yi), axis=0) / batch_size
      w_bce[digit] = (w_bce[digit][0] - lr * gradient, w_bce[digit][1] - lr * gradient_b)

### Testing the classifiers

In [11]:
for digit in range(10):
  loss = 0
  correct = 0
  
  for example in range(len(flat_x_test)):
      z = flat_x_test[example].dot(w_bce[digit][0]) + w_bce[digit][1]
      a = sigmoid(z)
      prediction = int(round(a[0]))
      loss += bce_loss(prediction, y_test_single_digit[digit][example])
      correct += 1 if y_test_single_digit[digit][example] == prediction else 0
        
  loss /= len(flat_x_test)
  accuracy = correct / len(flat_x_test)
  print("Classifier " + str(digit) + " | loss: " + str(loss) + " - accuracy: " + str(accuracy))

Classifier 0 | loss: 0.07199999956918797 - accuracy: 0.992
Classifier 1 | loss: 0.06119999956866659 - accuracy: 0.9932
Classifier 2 | loss: 0.1808999995744447 - accuracy: 0.9799
Classifier 3 | loss: 0.20609999957566064 - accuracy: 0.9771
Classifier 4 | loss: 0.15389999957314154 - accuracy: 0.9829
Classifier 5 | loss: 0.22409999957652907 - accuracy: 0.9751
Classifier 6 | loss: 0.11969999957149098 - accuracy: 0.9867
Classifier 7 | loss: 0.14039999957249027 - accuracy: 0.9844
Classifier 8 | loss: 0.34649999958243555 - accuracy: 0.9615
Classifier 9 | loss: 0.30419999958039445 - accuracy: 0.9662


### Using all classifiers to perform digit classification on test data

In [12]:
correct = 0

for example in range(len(flat_x_test)):
  y_predict = np.array([])  
  
  for classifier in range(10):
    z = flat_x_test[example].dot(w_bce[classifier][0]) + w_bce[classifier][1]
    a = sigmoid(z)
    y_predict = np.append(y_predict, a)
    
  if np.argmax(y_predict) == y_test[example]:
    correct += 1
    
print("Accuracy on test data : " + str(correct / len(flat_x_test)))

Accuracy on test data : 0.9161


# Problem 3
## Using softmax and categorical cross entropy loss

### Training the network

In [32]:
epochs = 10
lr = 0.01
batch_size = 32

for epoch in range(epochs):
  shuffled_indices = np.random.permutation(len(flat_x_train))
  flat_x_train_shuffled = flat_x_train[shuffled_indices]
  y_train_shuffled = y_train_cat[shuffled_indices]
  correct = 0
  loss = 0
  
  for i in range(0, len(flat_x_train), batch_size):
    xi = flat_x_train_shuffled[i:i+batch_size]
    yi = y_train_shuffled[i:i+batch_size]
      
    for j in range(batch_size):
      z = xi[j].dot(w_cce).reshape(10,1) + w_cce_b.reshape(10,1)
      a = softmax(z)
      correct += 1 if np.argmax(a) == np.argmax(yi[j]) else 0
      loss += cce_loss(a, yi[j].reshape(10,1))
      gradient = cce_gradient(a, xi[j], yi[j].reshape(10,1)) if j == 0 else gradient + cce_gradient(a, xi[j], yi[j].reshape(10,1))
      gradient_b = cce_gradient(a, 1, yi[j].reshape(10,1)) if j == 0 else gradient_b + cce_gradient(a, 1, yi[j].reshape(10,1))
    
    gradient = gradient / batch_size
    gradient_b = gradient_b / batch_size

    w_cce = w_cce - lr * gradient.T
    w_cce_b = w_cce_b - lr * gradient_b
    
  loss /= len(flat_x_train)
  accuracy = correct / len(flat_x_train)
  print("After epoch " + str(epoch + 1) + " | loss: " + str(loss) + " - accuracy: " + str(accuracy))

After epoch 1 | loss: 0.15518335550584886 - accuracy: 0.89695
After epoch 2 | loss: 0.1523986044335064 - accuracy: 0.8988333333333334
After epoch 3 | loss: 0.15000243973397903 - accuracy: 0.90055
After epoch 4 | loss: 0.1478709656101087 - accuracy: 0.9025333333333333
After epoch 5 | loss: 0.14606991348107756 - accuracy: 0.9030833333333333
After epoch 6 | loss: 0.14436018951092847 - accuracy: 0.9048333333333334
After epoch 7 | loss: 0.14279564525260527 - accuracy: 0.9062
After epoch 8 | loss: 0.14144549538561363 - accuracy: 0.90705
After epoch 9 | loss: 0.14018041823260582 - accuracy: 0.9083
After epoch 10 | loss: 0.13900374648994618 - accuracy: 0.9091833333333333


### Testing the network on test data

In [33]:
loss = 0
correct = 0

for example in range(len(flat_x_test)):
  z = flat_x_test[example].dot(w_cce).reshape(10,1) + w_cce_b.reshape(10,1)
  y_predict = softmax(z)
  correct += 1 if np.argmax(y_predict) == y_test[example] else 0
  loss += cce_loss(y_predict, y_test_cat[example].reshape(10,1))

loss /= len(flat_x_test)
accuracy = correct / len(flat_x_test)
print("loss: " + str(loss) + " - accuracy: " + str(accuracy))

loss: 0.13595054791523534 - accuracy: 0.9101


# Problem 4
## Reimplementing the network from Problem 3 entirely in Keras

### Setting up the network architecture

In [15]:
network = models.Sequential()
network.add(layers.Dense(10, activation='softmax', input_shape=(28 * 28,)))
network.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


### Compiling the network

In [0]:
network.compile(optimizer='sgd',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

### Training the network

In [17]:
epochs = 10
network.fit(flat_x_train, 
            y_train_cat, 
            epochs=epochs, 
            batch_size=32, 
            validation_data=(flat_x_test, y_test_cat))

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 8s 141us/step - loss: 0.7832 - acc: 0.8115 - val_loss: 0.4832 - val_acc: 0.8807
Epoch 2/10
60000/60000 [==============================] - 6s 101us/step - loss: 0.4576 - acc: 0.8797 - val_loss: 0.4020 - val_acc: 0.8939
Epoch 3/10
60000/60000 [==============================] - 6s 102us/step - loss: 0.4043 - acc: 0.8908 - val_loss: 0.3694 - val_acc: 0.9003
Epoch 4/10
60000/60000 [==============================] - 6s 100us/step - loss: 0.3776 - acc: 0.8960 - val_loss: 0.3506 - val_acc: 0.9043
Epoch 5/10
60000/60000 [==============================] - 6s 100us/step - loss: 0.3607 - acc: 0.9006 - val_loss: 0.3372 - val_acc: 0.9090
Epoch 6/10
60000/60000 [==============================] - 6s 100us/step - loss: 0.3487 - acc: 0.9025 - val_loss: 0.3299 - val_acc: 0.9096
Epoch 7/10
60000/60000 [==============================] - 6s 101us/step - 

# Problem 5
## Extending the network from Problem 4 by adding new features

### Methods for extracting features

In [0]:
def round_grey_values(original_image):
  return np.round(original_image / 255).astype(int)

# Get number of white regions for an image (Threshold = 100)
def num_white_regions(original_image):
  image = np.copy(original_image)
  num_regions = 0
  
  for i in range(len(image)):
    for j in range(len(image[i])):
      if image[i,j] == 1:
        num_regions += 1
        visit_region(image, i, j)
        
  return num_regions

# num_white_regions() helper function
def visit_region(image, i, j):
  if (i < 0 or j < 0 or i >= len(image)
      or j >= len(image[0]) or image[i,j] == 0):
    return
  
  image[i,j] = 0
  visit_region(image, i + 1, j)
  visit_region(image, i - 1, j)
  visit_region(image, i, j + 1)
  visit_region(image, i, j - 1)

def digit_width(image):
  min_x = len(image)
  max_x = 0
  
  for i in range(len(image)):
    min_x = min(np.argmax(image[i]), min_x)
    max_x = max(np.argmax(np.flip(image[i], 0)), max_x)
  
  return max_x - min_x

def digit_height(image):
  return digit_width(image.T)

def digit_area(image):
  return np.sum(image)

### Extract number of white regions from image

In [0]:
num_regions_train = []
num_regions_test = []

for image in range(len(x_train)):
  num_regions_train.append(num_white_regions(x_train[image]))
  
for image in range(len(x_test)):
  num_regions_test.append(num_white_regions(x_test[image]))
  
num_regions_train = np.array(num_regions_train)
num_regions_test = np.array(num_regions_test)

### Extract white regions, digit height, width and area from image

In [0]:
digit_width_train = []
digit_width_test = []
digit_height_train = []
digit_height_test = []
digit_area_train = []
digit_area_test = []

for image in range(len(x_train)):
  digit_width_train.append(digit_width(x_train[image]))
  digit_height_train.append(digit_height(x_train[image]))
  digit_area_train.append(digit_area(x_train[image]))
  
for image in range(len(x_test)):
  digit_width_test.append(digit_width(x_test[image]))
  digit_height_test.append(digit_height(x_test[image]))
  digit_area_test.append(digit_area(x_test[image]))
  
digit_width_train = np.array(digit_width_train)
digit_width_test = np.array(digit_width_test)

digit_height_train = np.array(digit_height_train)
digit_height_test = np.array(digit_height_test)

digit_area_train = np.array(digit_area_train)
digit_area_test = np.array(digit_area_test)

### Feature scale engineered data

In [0]:
# Find max and mins of train and test
num_regions_min = min(np.min(num_regions_train), np.min(num_regions_test))
num_regions_max = max(np.max(num_regions_train), np.max(num_regions_test))

digit_width_min = min(np.min(digit_width_train), np.min(digit_width_test))
digit_width_max = max(np.max(digit_width_train), np.max(digit_width_test))

digit_height_min = min(np.min(digit_height_train), np.min(digit_height_test))
digit_height_max = max(np.max(digit_height_train), np.max(digit_height_test))

digit_area_min = min(np.min(digit_area_train), np.min(digit_area_test))
digit_area_max = max(np.max(digit_area_train), np.max(digit_area_test))


# Use found max and mins to feature scale
num_regions_train = num_regions_train - num_regions_min
num_regions_train = num_regions_train / num_regions_max

num_regions_test = num_regions_test - num_regions_min
num_regions_test = num_regions_test / num_regions_max

digit_width_train = digit_width_train - digit_width_min
digit_width_train = digit_width_train / digit_width_max

digit_width_test = digit_width_test - digit_height_min
digit_width_test = digit_width_test / digit_height_max

digit_height_train = digit_height_train - digit_height_min
digit_height_train = digit_height_train / digit_height_max

digit_height_test = digit_height_test - digit_height_min
digit_height_test = digit_height_test / digit_height_max

digit_area_train = digit_area_train - digit_area_min
digit_area_train = digit_area_train / digit_area_max

digit_area_test = digit_area_test - digit_area_min
digit_area_test = digit_area_test / digit_area_max

### Stack flattened image on top of engineered features

In [0]:
engineered_features_train = np.array([num_regions_train, 
                                      digit_width_train, 
                                      digit_height_train, 
                                      digit_area_train])
engineered_features_test = np.array([num_regions_test, 
                                     digit_width_test, 
                                     digit_height_test, 
                                     digit_area_test])

In [0]:
flat_x_train2 = []
flat_x_test2 = []

for i in range(len(flat_x_train)):
  flat_x_train2.append(np.append(flat_x_train[i], engineered_features_train[:,i]))
  
for i in range(len(flat_x_test)):
  flat_x_test2.append(np.append(flat_x_test[i], engineered_features_test[:,i]))
  
flat_x_train2 = np.array(flat_x_train2)
flat_x_test2 = np.array(flat_x_test2)

### Setting up the network architecture

In [25]:
network2 = models.Sequential()
network2.add(layers.Dense(10, activation='softmax', input_shape=(28 * 28 + 4,)))
network2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 10)                7890      
Total params: 7,890
Trainable params: 7,890
Non-trainable params: 0
_________________________________________________________________


### Compiling the network

In [0]:
network2.compile(optimizer='sgd',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

### Training the network

In [27]:
epochs = 10
network2.fit(flat_x_train2, 
            y_train_cat, 
            epochs=epochs, 
            batch_size=32, 
            validation_data=(flat_x_test2, y_test_cat))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 6s 103us/step - loss: 0.7686 - acc: 0.8176 - val_loss: 0.4788 - val_acc: 0.8808
Epoch 2/10
60000/60000 [==============================] - 6s 100us/step - loss: 0.4550 - acc: 0.8815 - val_loss: 0.3984 - val_acc: 0.8975
Epoch 3/10
60000/60000 [==============================] - 6s 99us/step - loss: 0.4022 - acc: 0.8918 - val_loss: 0.3658 - val_acc: 0.9035
Epoch 4/10
60000/60000 [==============================] - 6s 100us/step - loss: 0.3756 - acc: 0.8976 - val_loss: 0.3466 - val_acc: 0.9068
Epoch 5/10
60000/60000 [==============================] - 6s 100us/step - loss: 0.3588 - acc: 0.9011 - val_loss: 0.3353 - val_acc: 0.9095
Epoch 6/10
60000/60000 [==============================] - 6s 100us/step - loss: 0.3470 - acc: 0.9038 - val_loss: 0.3252 - val_acc: 0.9119
Epoch 7/10
60000/60000 [==============================] - 6s 99us/step - loss: 0.3379 - acc: 0.9060 - val_loss: 0.3185 - va